In [3]:
!pip install qdrant_client langchain_huggingface langchain_community langchain_qdrant Pypdf openai langchain_core langchain

In [6]:
from qdrant_client import QdrantClient
from langchain_core.documents import Document
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_qdrant import QdrantVectorStore
import openai
import os

In [7]:
embed_model=HuggingFaceEmbeddings(model_name='BAAI/bge-small-en-v1.5')

<ipython-input-7-f9c56d997812>:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embed_model=HuggingFaceEmbeddings(model_name='BAAI/bge-small-en-v1.5')
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your setting

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/94.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
loader=PyPDFLoader("/content/National AI Policy Consultation Draft V1.pdf")
pages=loader.load()

In [9]:
pages[15]

Document(metadata={'source': '/content/National AI Policy Consultation Draft V1.pdf', 'page': 15}, page_content=" \n \n1 \n 4 Policy Directives  \nThe policy directives are minimalistic, focusing  on resolving issues and achieving targets set for stimulating \ngrowth in AI across the board. Empathizing with  the common  person's journey  for different aspects \nassociated with their socio -economic development and well -being in the current technological disruption \nis driven through the following developmental pillars.  \n4.1 1st Pillar: AI Market Enablement  \n4.1.1  National Artificial Intelligence Fund (NA IF)  \nGiven the evidence regarding the state of AI in Pakistan, the projected global outlook of AI in terms of its \nuse and market size, the impact of AI on the local ecosystem , and claiming its demographic share through \nresponsible use of data, the Ministry  of IT & Telecom through its underutilized resources and funds aims \nto establish a National AI Fund with following 

In [10]:
len(pages)

41

# TEXT SPLITTING

In [11]:
text_splitter=RecursiveCharacterTextSplitter(
    chunk_size=1500,
    chunk_overlap=150
)

# META DATA PREPROCESSING

In [12]:
from langchain.docstore.document import Document

In [13]:
doc_list=[]
for page in pages:
  pg_split=text_splitter.split_text(page.page_content)
  for pg_sub_split in pg_split:
    metadata={"source":"AI Policy","page_no":page.metadata["page"]+1}
    doc_string=Document(page_content=pg_sub_split,metadata=metadata)
    doc_list.append(doc_string)

In [14]:
 doc_list[10]

Document(metadata={'source': 'AI Policy', 'page_no': 6}, page_content='6 \n 1 Executive  Summary  \nPakistan has a unique opportunity to harness digital disruption by educating an eager young population \nthat can potentially propel the n ation onto a growth trajectory to sustain our future national \ncompetitiveness and improve the lives of citizens. Artificial Intelligence (AI) represents the next frontier of \ntechnological opportunities, and it has been widely proven and understood that the  collection, processing, \nuse, and exchange of data through automated/intelligent means would drive the entire society into the \nnext stage of its evolution which is unprecedented and requires a progressive , yet careful approach. So, \nafter a thorough analysi s of the global perspective and based on the evidence collected through more \nextensive consultations with the stakeholders, the Ministry of IT & Telecom has come to a much -desired \nconclusion that it needs to chalk out a development

In [16]:
qdrant_url='https://1d396cf1-958f-4af1-8f51-f7a2c358a330.europe-west3-0.gcp.cloud.qdrant.io:6333'
qdrant_api_key='ZQvSeTDrFwWIvDMndl8CQTINtwAypto_o4GHcrxHYK6IE-LzDbFGiA'
collection_name='AI_Policy'

In [17]:
qdrant=QdrantVectorStore.from_documents(
    doc_list,
    embed_model,
    url=qdrant_url,
    api_key=qdrant_api_key,
    collection_name=collection_name
)

# Query Vectore Store

In [21]:
query="what is AI policy for students"

result=qdrant.similarity_search(query,k=5)

In [22]:
result[3]

Document(metadata={'source': 'AI Policy', 'page_no': 5, '_id': 'c39041ed-6eee-45c9-bf05-b7fa9efecf37', '_collection_name': 'AI_Policy'}, page_content='5 \n Foreword  \nThrough its continued efforts toward  the early realization of the Digital Pakistan vision of the Government \nof Pakistan, the Ministry of IT & Telecom is committed to providing  its people with timely and equal access \nto opportunities by stimulating a culture of innovation through an overarching de velopmental agenda \norchestrated to embrace cutting -edge technologies such as Artificial Intelligence efficiently and \nresponsibly.   \nIn this regard, the  Artificial Intelligence  (AI) Policy  is a pivotal milestone for  transforming Pakistan into a  \nknowledge -based economy as it spells out a national strategy  to establish an ecosystem necessary for AI \nadoption by harnessing an agile framework for addressing different aspects of unique user journeys \nencompassing different market horizontals and industry vertic

In [23]:
result[1]

Document(metadata={'source': 'AI Policy', 'page_no': 27, '_id': '487eca83-cf0c-4cd6-9e65-e414624cbf93', '_collection_name': 'AI_Policy'}, page_content='policies.  \nIX. Develop regulation policies and standards for data -sharing among countries and lead multilateral \ndiplomatic efforts to arra nge such agreements.  \nX. Encourage local businesses to embrace new AI solutions and provide them with a platform for \ntechnical support and some incentives and regulations. Moreover, it should catalyze the creation \nof new businesses based on AI technology throu gh start -up funds and incubation centers . \nXI. Formulate policies to develop and maintain highly resilient cutting -edge computing, storage, and \nconnectivity infrastructure.  \nXII. Participate in international efforts to bring standardization in all aspects of AI, e.g. , data formats, \nnetwork and systems architecture, data , application integration protocols, requirements on test \ncases, and services.  \nXIII. Develop a data

In [24]:
result[0].page_content

'4.2.3  Algorithms, Data Science  & AI in Basic Education  \nI. Where the policy document emphasizes the funda mental  understanding and awareness of \npersonal data protection and AI, it also aims to stimulate  an incremental impact of AI on society \nright from the grassroots . Therefore, the policy has given equal importance to teach ing algorithms, \ndata science , and AI in basic STEM education . In this regard, it stipulates that. CoE-AI shall  hire a \nlocal/international consultant with expertise in high -tech curriculum development to develop a \nNational Curriculum in Algorithms, Data Sciences, AI , and Allied Technologies from the sixth to the  \ntwelfth standard.'

In [27]:
!pip install -qU langchain-pinecone pinecone-notebooks

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.8/244.8 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.4/85.4 kB 4.4 MB/s eta 0:00:00


In [32]:
pinecone_api_key='a335fe92-1e88-4743-8222-d5d9e7bd984d'
index_name='demo-vectorestore'

In [33]:
from langchain_pinecone import PineconeVectorStore as lang_pinecone
os.environ['PINECONE_API_KEY']=pinecone_api_key

In [34]:
vectore=lang_pinecone.from_documents(
    doc_list,
    embed_model,
    index_name=index_name
)

In [36]:
query="what is Ai policy for students"

In [37]:
pinecone_result=vectore.similarity_search(query,k=5)

In [38]:
pinecone_result

[Document(id='c7c96148-47b8-4cf0-92a4-be45cd0d33eb', metadata={'page_no': 25.0, 'source': 'AI Policy'}, page_content='4.2.3  Algorithms, Data Science  & AI in Basic Education  \nI. Where the policy document emphasizes the funda mental  understanding and awareness of \npersonal data protection and AI, it also aims to stimulate  an incremental impact of AI on society \nright from the grassroots . Therefore, the policy has given equal importance to teach ing algorithms, \ndata science , and AI in basic STEM education . In this regard, it stipulates that. CoE-AI shall  hire a \nlocal/international consultant with expertise in high -tech curriculum development to develop a \nNational Curriculum in Algorithms, Data Sciences, AI , and Allied Technologies from the sixth to the  \ntwelfth standard.'),
 Document(id='e7c1ebf0-9e3c-4dca-b39b-4b9e17e8fbc7', metadata={'page_no': 27.0, 'source': 'AI Policy'}, page_content='policies.  \nIX. Develop regulation policies and standards for data -sharing a

In [39]:
pinecone_result[1].page_content

'policies.  \nIX. Develop regulation policies and standards for data -sharing among countries and lead multilateral \ndiplomatic efforts to arra nge such agreements.  \nX. Encourage local businesses to embrace new AI solutions and provide them with a platform for \ntechnical support and some incentives and regulations. Moreover, it should catalyze the creation \nof new businesses based on AI technology throu gh start -up funds and incubation centers . \nXI. Formulate policies to develop and maintain highly resilient cutting -edge computing, storage, and \nconnectivity infrastructure.  \nXII. Participate in international efforts to bring standardization in all aspects of AI, e.g. , data formats, \nnetwork and systems architecture, data , application integration protocols, requirements on test \ncases, and services.  \nXIII. Develop a data -sharing framework  and use AI algorithms consistent with social, cultural, and \nreligious norms and internatio nal guidelines.  \nXIV. A governance 